In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("log2.csv")
x = df.drop(columns=['Source Port','Destination Port','NAT Source Port','NAT Destination Port','Action'])
y = df['Action']
N_FEATURES = x.shape[1]
y = pd.get_dummies(y)*1
N_CLASSES = y.shape[1]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.1)
max = x_train.max()
min = x_train.min()
x_train = (max - x_train) / (max - min)
x_test = (max - x_test) / (max - min)
dnn = tf.keras.models.Sequential()
dnn.add(tf.keras.layers.Dense(5, activation='relu', input_shape=(N_FEATURES,)))
dnn.add(tf.keras.layers.Dropout(0.2))
dnn.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))
dnn.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
dnn.fit(x=x_train, y=y_train, epochs=10, batch_size=64)
dnn.evaluate(x=x_test, y=y_test)

Epoch 1/10
922/922 [==============================] - 2s 1ms/step - loss: 1.2502 - accuracy: 0.5317 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/10
922/922 [==============================] - 2s 2ms/step - loss: 1.0226 - accuracy: 0.5746 - precision: 0.5746 - recall: 0.4250
Epoch 3/10
922/922 [==============================] - 1s 2ms/step - loss: 0.9960 - accuracy: 0.5746 - precision: 0.5746 - recall: 0.5746
Epoch 4/10
922/922 [==============================] - 1s 1ms/step - loss: 0.9907 - accuracy: 0.5746 - precision: 0.5746 - recall: 0.5746
Epoch 5/10
922/922 [==============================] - 2s 2ms/step - loss: 0.9864 - accuracy: 0.5746 - precision: 0.5746 - recall: 0.5746
Epoch 6/10
922/922 [==============================] - 1s 2ms/step - loss: 0.9830 - accuracy: 0.5746 - precision: 0.5746 - recall: 0.5746
Epoch 7/10
922/922 [==============================] - 1s 1ms/step - loss: 0.9792 - accuracy: 0.5746 - precision: 0.5746 - recall: 0.5746
Epoch 8/10
922/922 [=============

[0.9711974263191223,
 0.5720170736312866,
 0.5720170736312866,
 0.5720170736312866]

In [2]:
# Saliency Map
n_instance_test = x_test.shape[0]
S = [] # S \in R^{n_instances \times n_classes \times n_features}
for i in range(n_instance_test):
    s_i = []
    x_i = tf.Variable(x_test.values[i].reshape(1,7))
    for r in range(N_CLASSES):
        with tf.GradientTape() as tape:
            tape.watch(x_i)
            logits = dnn(x_i)
            class_index = r
            score_r = logits[:, class_index]
            gradient_r = tape.gradient(score_r, x_i)
        s_i.append(gradient_r[0].numpy().tolist())
    S.append(s_i)
S = np.array(S)
np.save('saliency_x_test.npy',S)

In [4]:
S=np.load('saliency_x_test.npy')

In [5]:
S.shape

(6554, 4, 7)